In [ ]:
from IPython.display import HTML
from matplotlib.animation import FuncAnimation
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import numpy as np
import sympy as sm
import sympy.physics.mechanics as me

me.init_vprinting(use_latex='mathjax')

In [ ]:
q1, q2, q3 = me.dynamicsymbols('q1, q2, q3')
u1, u2, u3 = me.dynamicsymbols('u1, u2, u3')
la, lb, lc, ln = sm.symbols('l_a, l_b, l_c, l_n')
m, g = sm.symbols('m, g')
t = me.dynamicsymbols._t

p = sm.Matrix([la, lb, lc, ln, m, g])

q = sm.Matrix([q1])
qr = sm.Matrix([q2, q3])
qN = q.col_join(qr)

u = sm.Matrix([u1])
ur = sm.Matrix([u2, u3])
uN = u.col_join(ur)

qdN = qN.diff(t)
ud = u.diff(t)

p, q, qr, qN, u, ur, uN, qdN, ud

In [ ]:
N = me.ReferenceFrame('N')
A = me.ReferenceFrame('A')
B = me.ReferenceFrame('B')
C = me.ReferenceFrame('C')

A.orient_axis(N, q1, N.z)
B.orient_axis(A, q2, A.z)
C.orient_axis(B, q3, B.z)

P1 = me.Point('P1')
P2 = me.Point('P2')
P3 = me.Point('P3')
P4 = me.Point('P4')

P2.set_pos(P1, la*A.x)
P3.set_pos(P2, lb*B.x)
P4.set_pos(P3, lc*C.x)

In [ ]:
loop = P4.pos_from(P1) - ln*N.x

fh = sm.Matrix([loop.dot(N.x), loop.dot(N.y)])
fh = sm.trigsimp(fh)
fh

In [ ]:
fk = uN - qdN
fk

In [ ]:
Mk = fk.jacobian(qdN)
gk = uN
qd_sol = -Mk.LUsolve(gk)
qd_repl = dict(zip(qdN, qd_sol))
qd_repl

In [ ]:
fhd = fh.diff(t).xreplace(qd_repl)
fhd

In [ ]:
me.find_dynamicsymbols(fhd)

In [ ]:
Mhd = fhd.jacobian(ur)
ur_zero = {uri: 0 for uri in ur}
ghd = fhd.xreplace(ur_zero)
ur_sol = -Mhd.LUsolve(ghd)
ur_repl = dict(zip(ur, ur_sol))

In [ ]:
ur_repl

In [ ]:
A.set_ang_vel(N, u1*N.z)
B.set_ang_vel(A, ur_repl[u2]*A.z)
C.set_ang_vel(B, ur_repl[u3]*B.z)

In [ ]:
P1.set_vel(N, 0)
P2.v2pt_theory(P1, N, A)
P3.v2pt_theory(P2, N, B)
P4.v2pt_theory(P3, N, C)

In [ ]:
me.find_dynamicsymbols(P4.vel(N), reference_frame=N)

In [ ]:
gk = gk.xreplace(ur_repl)

In [ ]:
me.find_dynamicsymbols(gk)

In [ ]:
R_P2 = -m*g*N.y
R_P3 = -m*g*N.y

In [ ]:
F1 = P2.vel(N).diff(u1, N).dot(R_P2) + P3.vel(N).diff(u1, N).dot(R_P3)
F1

In [ ]:
me.find_dynamicsymbols(P3.acc(N).xreplace(qd_repl).xreplace(ur_repl), reference_frame=N)

In [ ]:
N_a_P2 = P2.acc(N).xreplace(qd_repl).xreplace(ur_repl)
N_a_P3 = P3.acc(N).xreplace(qd_repl).xreplace(ur_repl)

In [ ]:
Rs_P2 = -m*N_a_P2
Rs_P3 = -m*N_a_P3

In [ ]:
F1s = P2.vel(N).diff(u1, N).dot(Rs_P2) + P3.vel(N).diff(u1, N).dot(Rs_P3)

In [ ]:
me.find_dynamicsymbols(F1)

In [ ]:
me.find_dynamicsymbols(F1s)

In [ ]:
fd = F1 + F1s

In [ ]:
fh

In [ ]:
fk.xreplace(ur_repl)

In [ ]:
p_vals = np.array([
    0.8,  # la [m]
    2.0,  # lb [m]
    1.0,  # lc [m]
    2.0,  # ln [m]
    1.0,  # m [kg]
    9.81,  # g [m/s^2]
])


In [ ]:
q1_0 = np.deg2rad(10.0)
u1_0 = 0.0

In [ ]:
from scipy.optimize import fsolve

In [ ]:
fsolve?

In [ ]:
eval_fh = sm.lambdify((qN, p), fh)

In [ ]:
eval_fh([1.0, 2.0, 3.0], p_vals)

In [ ]:
def fsolve_fh(qr, q1, p):
    return np.squeeze(eval_fh(np.hstack((q1, qr)), p))

In [ ]:
fsolve_fh([2.0, 3.0], 1.0, p_vals)

In [ ]:
qr_guess = np.deg2rad([10.0, -150.0])

In [ ]:
q2_0, q3_0 = fsolve(fsolve_fh, qr_guess, args=(q1_0, p_vals), xtol=1e-14)

In [ ]:
q1_0, q2_0, q3_0

In [ ]:
eval_fh([q1_0, q2_0, q3_0], p_vals)

In [ ]:
me.find_dynamicsymbols(fd)

In [ ]:
Md = fd.diff(u1.diff())
gd = fd.xreplace({u1.diff(): 0})
u1_sol = -gd/Md
eval_u1 = sm.lambdify((u1, qN, p), u1_sol)

In [ ]:
u1d_0 = eval_u1(u1_0, [q1_0, q2_0, q3_0], p_vals)
u1d_0

In [ ]:
q1_0, q2_0, q3_0, u1_0, u1d_0

In [ ]:
from scikits.odes import dae

In [ ]:
dae?

In [ ]:
u1d = sm.symbols('u1d')
eval_fd = sm.lambdify((qN, u1, u1d, p), fd.xreplace({u1.diff(): u1d}))
eval_fd([q1_0, q2_0, q3_0], 1.0, 3.0, p_vals)

In [ ]:
eval_fh?

In [ ]:
def eval_eom(t, x, xd, residual, p):
    qN = x[:3]
    u1 = x[3]
    q1d = xd[0]
    u1d = xd[3]
    
    residual[0:2] = eval_fh(qN, p).squeeze()  # shape(2,)
    residual[2] = u1 - q1d  # shape(1,)
    residual[3] = eval_fd(qN, u1, u1d, p).squeeze()  # shape(1,)

In [ ]:
res = np.ones(4)
res

In [ ]:
eval_eom(1.0, # t
         [q1_0, q2_0, q3_0, u1_0], # x: q1, q2, q3, u1
         [0.0, 0.0, 0.0, u1d_0], # xd: q1d, q2q, q3d, u1d
         res,
         p_vals)

In [ ]:
res

In [ ]:
solver = dae('ida',  # which solver to use
             eval_eom,
             rtol=1e-10,
             atol=1e-10,
             algebraic_vars_idx=[0, 1],
             user_data=p_vals,
             old_api=False)  # gives us new output format

In [ ]:
x0 = np.array([q1_0, q2_0, q3_0, u1_0])
xd0 = np.array([0.0, 0.0, 0.0, u1d_0])
ts = np.linspace(0.0, 4.0, num=401)

In [ ]:
solution = solver.solve(ts, x0, xd0)

In [ ]:
solution.values.y

In [ ]:
fig, ax = plt.subplots()

ax.plot(solution.values.t, solution.values.y)

In [ ]:
def eval_constraints(xs, p):
    """Returns the value of the left hand side of the holonomic constraints
    at each time instance.

    Parameters
    ==========
    xs : ndarray, shape(n, 4)
        States at each of n time steps.
    p : ndarray, shape(6,)
        Constant parameters.

    Returns
    =======
    con : ndarray, shape(n, 2)
        fh evaluated at each xi in xs.

    """
    con = []
    for xi in xs:  # xs is shape(n, 4)
        con.append(eval_fh(xi[:3], p).squeeze())
    return np.array(con)

In [ ]:
con = eval_constraints(solution.values.y, p_vals)

In [ ]:
def plot_results(ts, xs, con):
    """Returns the array of axes of a 4 panel plot of the state trajectory
    versus time.

    Parameters
    ==========
    ts : array_like, shape(n,)
       Values of time.
    xs : array_like, shape(n, 4)
       Values of the state trajectories corresponding to ``ts`` in order
       [q1, q2, q3, u1].
    con : array_like, shape(n, 2)
       x and y constraint residuals of P4 at each time in ``ts``.

    Returns
    =======
    axes : ndarray, shape(3,)
       Matplotlib axes for each panel.

    """
    fig, axes = plt.subplots(3, 1, sharex=True)

    fig.set_size_inches((10.0, 6.0))

    axes[0].plot(ts, np.rad2deg(xs[:, :3]))  # q1(t), q2(t), q3(t)
    axes[1].plot(ts, np.rad2deg(xs[:, 3]))  # u1(t)
    axes[2].plot(ts, np.squeeze(con))  # fh(t)

    axes[0].legend(['$q_1$', '$q_2$', '$q_3$'])
    axes[1].legend(['$u_1$'])
    axes[2].legend([r'$\cdot\hat{n}_x$', r'$\cdot\hat{n}_y$'])

    axes[0].set_ylabel('Angle [deg]')
    axes[1].set_ylabel('Angular Rate [deg/s]')
    axes[2].set_ylabel('Distance [m]')
    axes[2].set_xlabel('Time [s]')

    fig.tight_layout()

    return axes


In [ ]:
plot_results(solution.values.t, solution.values.y, con)

In [ ]:
coordinates = P2.pos_from(P1).to_matrix(N)
for point in [P3, P4, P1, P2]:
    coordinates = coordinates.row_join(point.pos_from(P1).to_matrix(N))
eval_point_coords = sm.lambdify((qN, p), coordinates)

In [ ]:
def setup_animation_plot(ts, xs, p):
    """Returns objects needed for the animation.

    Parameters
    ==========
    ts : array_like, shape(n,)
       Values of time.
    xs : array_like, shape(n, 4)
       Values of the state trajectories corresponding to ``ts`` in order
       [q1, q2, q3, u1].
    p : array_like, shape(6,)

    """

    x, y, z = eval_point_coords(xs[0, :3], p)

    fig, ax = plt.subplots()
    fig.set_size_inches((10.0, 10.0))
    ax.set_aspect('equal')
    ax.grid()

    lines, = ax.plot(x, y, color='black',
                     marker='o', markerfacecolor='blue', markersize=10)

    title_text = ax.set_title('Time = {:1.1f} s'.format(ts[0]))
    ax.set_xlim((-1.0, 3.0))
    ax.set_ylim((-1.0, 1.0))
    ax.set_xlabel('$x$ [m]')
    ax.set_ylabel('$y$ [m]')

    return fig, ax, title_text, lines

In [ ]:
def animate_linkage(ts, xs, p):
    """Returns an animation object.

    Parameters
    ==========
    ts : array_like, shape(n,)
    xs : array_like, shape(n, 4)
       x = [q1, q2, q3, u1]
    p : array_like, shape(6,)
       p = [la, lb, lc, ln, m, g]

    """
    # setup the initial figure and axes
    fig, ax, title_text, lines = setup_animation_plot(ts, xs, p)

    # precalculate all of the point coordinates
    coords = []
    for xi in xs:
        coords.append(eval_point_coords(xi[:3], p))
    coords = np.array(coords)

    # define the animation update function
    def update(i):
        title_text.set_text('Time = {:1.1f} s'.format(ts[i]))
        lines.set_data(coords[i, 0, :], coords[i, 1, :])

    # close figure to prevent premature display
    plt.close()

    # create and return the animation
    return FuncAnimation(fig, update, len(ts))


In [ ]:
xs = solution.values.y
fps = 100
HTML(animate_linkage(ts, xs, p_vals).to_jshtml(fps=fps))